In [ ]:
%pip install langchain langchain_openai langchain-community --upgrade

In [3]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

In [9]:
function = lambda x: x + 1
function(2)

3

In [4]:
from langchain_core.runnables import RunnableLambda

print(type(RunnableLambda(lambda x: x + 1))) # <class 'langchain.schema.runnable.RunnableLambda'>

<class 'langchain_core.runnables.base.RunnableLambda'>


In [5]:
chain = RunnableLambda(lambda x: x + 1) 

In [11]:
chain

RunnableLambda(lambda x: x + 1)

In [10]:
chain.invoke(1)

2

In [14]:
# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | (lambda x: x * 2)

print(type(sequence)) # <class 'langchain.schema.runnable.RunnableSequence'>
print('\n\n---')
print(sequence.invoke(1)) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

<class 'langchain_core.runnables.base.RunnableSequence'>


---
4


[4, 6, 8]

## RunnableParallel

The `RunnableParallel`, allows for multiple runnables to be invoked in parallel, construct using a dictionary of runnables to invoke in parallel.

In [ ]:
# A sequence that contains a RunnableParallel constructed using a dict literal
# The first runnable value goes to the "mul_2" key, and the second goes to the "mul_5" key
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_5": RunnableLambda(lambda x: x * 5),
}
sequence.invoke(1)  # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

In [16]:
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
} | RunnableLambda(lambda x: x['mul_2'] + x['mul_5'])
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

14

In [17]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel({
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
})

# This is a dictionary, however it will be composed with other runnables when used in a sequence:
parallel_two = {
    'mul_2': RunnableLambda(lambda x: x['input_one'] * 2),
    'mul_5': RunnableLambda(lambda x: x['input_two'] * 5)
}

print(type(parallel)) # <class 'langchain.schema.runnable.RunnableParallel'>
print(type(parallel_two)) # <class 'dict'>

<class 'langchain_core.runnables.base.RunnableParallel'>
<class 'dict'>


In [18]:
chain = parallel | RunnableLambda(lambda x: x['mul_2'] + x['mul_5']) 
chain.invoke(5)

35

In [19]:
second_chain = parallel_two | RunnableLambda(lambda x: x['mul_2'] + x['mul_5']) 
second_chain.invoke({'input_one': 5, 'input_two': 10})

60

In [20]:
parallel = RunnableParallel({
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
})

# This is bad practice:
test = lambda x : x + 1  | parallel
print(test)
test.invoke(5)

<function <lambda> at 0x0000027C1148A660>


AttributeError: 'function' object has no attribute 'invoke'

In [21]:
# This is good practice:
test = RunnableLambda(lambda x: x + 1) | parallel
print(test)
test.invoke(5)

first=RunnableLambda(lambda x: x + 1) middle=[] last={
  mul_2: RunnableLambda(...),
  mul_5: RunnableLambda(...)
}


{'mul_2': 12, 'mul_5': 30}

In [22]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel({
    'item_one': RunnableLambda(lambda x: f"Hello {x['name']} "),
    'item_two': RunnableLambda(lambda x: 'Welcome to the World!')
})


In [24]:
def combine(x):
    return x['item_one'] + x['item_two']

In [26]:
parallel_chain_example = parallel | combine
parallel_chain_example.invoke({'name': "Marcell"}) 

'Hello Marcell Welcome to the World!'

In [27]:
lambda_example = RunnableLambda(lambda x: {'item_one': 'Hello ', 'item_two': 'World'})
lambda_chain_example = lambda_example | combine
lambda_chain_example.invoke({})

'Hello World'